In [19]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm

# Sparse User Item Matrix
We choose 5 collections which have approximately 3,000 unique items in 10,000 transactions.

In [4]:
# Load data
df_bayc = pd.read_csv("data/BAYC.csv")
df_meebits = pd.read_csv("data/MEEBITS.csv")
df_doodles = pd.read_csv("data/DOODLES.csv")
df_azuki = pd.read_csv("data/AZUKI.csv")
df_coolcats = pd.read_csv("data/COOLCATS.csv")

In [36]:
collections = [df_bayc, df_meebits, df_doodles, df_azuki, df_coolcats]
coll_name = ["Bayc", "Meebits", "Doodles", "Azuki", "Coolcats"]

In [101]:
df_bayc

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,tokenID,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,14796711,1652847797,0xd53ac790ef514be48efd810b5aa8aed9148d0acb8c8a...,88,0xfe96291e142addf3f1a7c52269bd2654784edc7eefb6...,0x2a59d2927541d17c0fa19140703ecb4b697b765d,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x710f6e3bfc837f451efb2955feedaf8a601bbfb6,321,BoredApeYachtClub,BAYC,0,55,124599,27839268413,114999,3967099,deprecated,2
1,14796711,1652847797,0x95abb416954e9dab82609fc695f47494c615102c80e3...,51,0xfe96291e142addf3f1a7c52269bd2654784edc7eefb6...,0xa282017d04148a8f5623461838aef2dc58054a72,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xa8340d926c0ea053b5d8a0cf53aacf4ee4120f86,4785,BoredApeYachtClub,BAYC,0,45,124599,27839268413,110199,3353838,deprecated,2
2,14796615,1652846370,0x3d4ecde65e80f1a876e443fc6c294459b7200ad17ae0...,44,0x3b469831dc7ccc7dfc6c2f0549e2bfbdcf82d251d235...,0x9d11f86874198590c6e168e02000c9da8ffd9baf,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xa282017d04148a8f5623461838aef2dc58054a72,4785,BoredApeYachtClub,BAYC,0,50,124587,25101927609,114987,3923802,deprecated,98
3,14796555,1652845681,0x1125d8dc789b62b26a79c8550a64fb5ca3537119ccfd...,334,0xc5d93c96cdef5f69d508d86604efbaa9f429a7af6808...,0x13d8faf4a690f5ae52e2d2c52938d1167057b9af,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xa742203fe554fff075077b749f5402c62fc01bba,9355,BoredApeYachtClub,BAYC,0,106,820762,30482663141,508585,5090765,deprecated,158
4,14796536,1652845309,0x5a57f01de114d6c2e10683aa5b608a3e9a50882f97c5...,464,0x855e912a10415285e858ad4d82a588920a08520c5d13...,0x6449bbb550f1049236d803f0e43759999b9a2bee,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x13d8faf4a690f5ae52e2d2c52938d1167057b9af,9355,BoredApeYachtClub,BAYC,0,448,1526704,24456836748,1003215,27532255,deprecated,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,14448511,1648118687,0x9c5b4b8d2fc6054a6709dce05df333f3883804157dce...,10,0x61732c7d92ab2231fb72c88f5db60d29b39f52c368db...,0xe33a9a784286dc93b4fc277e1e426bb871d80566,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x9b72ca489d6f764f1c298a715eb58502a0c07e62,3052,BoredApeYachtClub,BAYC,0,126,385479,24494122298,277139,28785810,deprecated,348202
9996,14448471,1648118272,0x94f26f5d78f21149cbdfca62c2f0ffd571b34b028d51...,13,0x93b4597f10eb8de41f809917c88330361331ed156177...,0x8cf47f9f3b1d760352adc855da815ae888eba8ba,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xefc0e97209b2686dfa834c10eed7bdfe80553457,6521,BoredApeYachtClub,BAYC,0,190,358890,26822727118,255361,13382248,deprecated,348242
9997,14448299,1648116111,0x5821c3b4684029bc7dca7c819daac45e3af21cc0fb92...,205,0x3c10158bd38164462739851753e122efc34e26894714...,0x0f6367cb7264a76967e26bc6b4b8df5c8e9118c0,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0xe5f8fd8830aba0fdc7c9181dcfc92b491033297a,8752,BoredApeYachtClub,BAYC,0,96,358890,33415952630,255361,5057345,deprecated,348414
9998,14448295,1648116071,0x67eb7271b0ffffe5c2f873a5abff4ebd7bf7503b4a15...,154,0x515acb1d3cef3b29e673146f24175a1ea3dbbaecb70d...,0x0c9a83120e744533fc78c02aca5d25784ca3825d,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x82e63a0fa475e1f11bad26daea4f0872ad89c065,4704,BoredApeYachtClub,BAYC,0,21,124599,37233036053,110199,1218985,deprecated,348418


### Helper Function - user-item dictonary function
----

In [33]:
def get_user_item_dict(df, name):
    """
    generates a user-item dictionary for one collection
    
    INPUT
        df: (pandas DataFrame) csv file
        name: (str) collection name
    OUTPUT
        user_item: (dictionary)
    """
    from tqdm import tqdm
    user_item = defaultdict(list)
    user_i = []
    
    items = df["tokenID"].unique()
    for i in tqdm(items):
        user_i = []
        df_i = df[df["tokenID"] == i]
        user_i.extend(df_i["from"].unique())
        user_i.extend(df_i["to"].unique())
        asset_name = name + "/" + str(i)
        user_item[asset_name] = user_i
    return user_item

____

In [38]:
user_item_dict = defaultdict(list)

for nft, name in zip(collections, coll_name):
    user_item_dict.update(get_user_item_dict(nft, name))

100%|██████████| 3858/3858 [00:04<00:00, 905.81it/s] 


In [72]:
row = set()

col = list(user_item_dict.keys())
row.update(*user_item_dict.values())
row = list(row)

In [77]:
df_user_item = pd.DataFrame(index=row, columns=col)

In [78]:
df_user_item

,Bayc/321,Bayc/4785,Bayc/9355,Bayc/7048,Bayc/4194,Bayc/956,Bayc/4263,Bayc/3428,Bayc/6380,Bayc/3168,...,Coolcats/1556,Coolcats/1248,Coolcats/5955,Coolcats/5156,Coolcats/1957,Coolcats/2725,Coolcats/6855,Coolcats/6033,Coolcats/5952,Coolcats/336
0x6c05da12487308ca8824c3741b3d6f89fc4b512b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x6d91a43df4dae0af4b213dc63699ac482079841e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x7d8f76231883c9dee51c2eb6c53acc58b042497d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x71b68ca778e1329086147f33b0f1e51d785d807a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0xe887f13bfc9a33ca3de5a5ceb2ff3126f45b8100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0xf73bbbd0235f6974fa281e5357fdce2a70c93f4b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0x7c866452ef99f82e900dd3d35f0ad1f1e3e10b11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0xc803698a4be31f0b9035b6eba17623698f3e2f82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0xefae1190444aae8cff4915eb7d5054bfcdfedce6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for item in tqdm(user_item_dict.keys()):
    users = user_item_dict[item]
    for u in users:
        df_user_item[item][u] = 1

In [95]:
df_user_item = df_user_item.fillna(0)

In [ ]:
df_user_item.to_csv("data/sparse_UserItemMatrix.csv")

In [100]:
df_user_item[:20]

,Bayc/321,Bayc/4785,Bayc/9355,Bayc/7048,Bayc/4194,Bayc/956,Bayc/4263,Bayc/3428,Bayc/6380,Bayc/3168,...,Coolcats/1556,Coolcats/1248,Coolcats/5955,Coolcats/5156,Coolcats/1957,Coolcats/2725,Coolcats/6855,Coolcats/6033,Coolcats/5952,Coolcats/336
0x6c05da12487308ca8824c3741b3d6f89fc4b512b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x6d91a43df4dae0af4b213dc63699ac482079841e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x7d8f76231883c9dee51c2eb6c53acc58b042497d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x71b68ca778e1329086147f33b0f1e51d785d807a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0xe887f13bfc9a33ca3de5a5ceb2ff3126f45b8100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x3a9f45ac308ccc0a1a48b0f9e2f8ce859a0039ea,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0xcec2baca25afaf9bbecfd9a800ca2c71576cbf9f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0xa2828a232b24bc5b6510069473d363fd6f43cc45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x11ad632cf28ee4559bddf3d4dc2019e78eb7ec3c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0xdd89b924ba6a7d5029b1a3a76f363d1a571e4836,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
